## Tensorflow TF_data
- Data Generator와 같은 것
- 장점 : 좀더 커스터마이징이 좋음
- 단점 : 복잡하고 좀더 어려움

In [1]:
import os
from glob import glob

import numpy as np
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
%matplotlib inline

## Data Preprocess
train_paths = glob('../dataset/cifar/train/*.png')
test_paths = glob('../dataset/cifar/test/*.png')

path = train_paths[3]
path

In c:\users\urse\anaconda3\envs\untitled\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In c:\users\urse\anaconda3\envs\untitled\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In c:\users\urse\anaconda3\envs\untitled\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In c:\users\urse\anaconda3\envs\untitled\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In c:\users\urse\anaconda3\envs\untitled

'../dataset/cifar/train\\10002_frog.png'

## Hyperparameter Tunning

In [2]:
num_epochs=10
batch_size=32

learning_rate = 0.001
dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3,3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size = (2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs = net, name = 'BasicCNN')
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate),
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

print(model.summary())

Model: "BasicCNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0  

In [4]:
# class 이름 반환
def get_class_name(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    return lbl_name

# 전체 train 데이터 class 이름들 얻는 법
class_names = [get_class_name(path) for path in train_paths]
print(len(class_names))

classes = tf.unique(class_names).y.numpy()
print(classes)

50000
[b'frog' b'automobile' b'ship' b'cat' b'deer' b'airplane' b'truck' b'dog'
 b'bird' b'horse']


In [5]:
# 클래스 이름을 통해 onehot encoding 된 label 값들 반환
def get_onehot_encoding(label_name):
    class_list = tf.unique(class_names).y
    onehot_encoding = tf.cast(class_list == label_name, tf.uint8) # label_name 과 같으면 tf.cast uint8로
    return onehot_encoding

# dataset 읽어오는 것
def read_dataset(path):
    # read image
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.
    
    # read label
    class_name = get_class_name(path) # path이미지의 class_name 
    label = get_onehot_encoding(class_name)    
    return image, label

# image Augmentation 해주는것 (랜덤으로 좌우, 상하)
def image_preprocess(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image, label

# image prepreocess 연습
image, label = read_dataset(path)
print(image.shape, label.shape)
# transformed, label = image_preprocess(image, label)
# print(transformed.shape)



(32, 32, 3) (10,)


In [6]:
# train_dataset generator
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths) # image data 연결고리 지정
train_dataset = train_dataset.map(read_dataset) # image, label 읽어오는 함수
train_dataset = train_dataset.map(image_preprocess) # Data Augmentaton 함수
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size = len(train_paths))
train_dataset = train_dataset.repeat()

# test dataset generator
test_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
test_dataset = test_dataset.map(read_dataset)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.shuffle(buffer_size = len(train_paths))
test_dataset = test_dataset.repeat()

## Training

In [ ]:
steps_for_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size 

model.fit_generator(
    train_dataset,
    steps_per_epoch = steps_for_epoch,
    validation_data = test_dataset,
    validation_steps = validation_steps,
    epochs = num_epochs
)

Epoch 1/10
